In [4]:
#Web scraping for Indeed
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd

#Generating a url for job location and position
def get_url(position,location):
    position=position.replace(' ','-')
    location=location.replace(' ','-')
    template='https://malaysia.indeed.com/jobs?q={}&l={}'
    url=template.format(position,location)
    return url

#Fetching job
def get_jobs(card):
    job_name=card.find('h2','jobTitle').text
    jobURL= 'https://malaysia.indeed.com/' + card.get('href')
    job_url=jobURL.replace('/rc/clk?','viewjob?')
    company_name=card.find('span','companyName').text.strip()
    job_location=card.find('div','companyLocation').text.strip()
    try:
        summary = ''
        tag_str = ''
    
        job_summary=card.find('div','job-snippet')
        if job_summary.text == '':
            summary = 'Summary Not Available'
        else:
            for tag in job_summary('li'):
                tag_str = tag.text
                summary = summary + tag_str

    except AttributeError:
        summary = 'Summary not available'
    job_salary=card.find_all('div','heading6 tapItem-gutter metadataContainer')
    try:
        job_salary =card.find('span','salary-snippet')
        salary = job_salary.text.strip()
    except AttributeError:
        salary= 'N/A'
    postDATE=card.find('span','date').text.strip()
    N=int(''.join(filter(str.isdigit, postDATE)))
    post_date = datetime.now().date() - timedelta(days=N)
    today=datetime.today().strftime('%Y-%m-%d')
    
    job_info =(job_name, job_url, company_name, job_location, summary, salary, post_date, today)
    return job_info

def main(position,location):
    records=[]
    
    url=get_url(position,location)
    
    response=requests.get(url)

#Creating BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')
    cards=soup.find_all('a',class_='tapItem')
    for everyCard in cards:
        jobDetails=get_jobs(everyCard)
        records.append(jobDetails)
        
#Using pandas dataframe on the job information in a CSV file
    col=['Job_name','Job_url','Company_name','Job_location','Job_summary','Job_salary','Post_date','Today_date']
    
    indeed_data=pd.DataFrame(records,columns=col)    
    indeed_data.to_csv('D:\\joblisting.csv')

In [5]:
main('web developer','kuala lumpur')